Se analiza un DataSet con características de autos donde se pretender realizar una predicción a través del algoritmo de Árboles de Decisión para saber si es conveniente comprar un auto o no en base a su descripción.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('car.csv') #Lectura del archivo
df.columns = ['price', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'decision'] #Defino los nombres de las columnas
df.head() #Muestro los primeros 5 registros

,price,maint,doors,persons,lug_boot,safety,decision
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc


Se describen las siguientes columnas:  
-   Precio del auto (**price**)  
-   Costo de mantenimiento (**maint**)  
-   Número de puertas (**doors**)  
-   Capacidad de personas (**persons**)  
-   Tamaño del maletero (**lug_boot**)  
-   Seguridad estimada del coche (**safety**)  
-   Decisión (**decision**)  

Esta última columna determina si es el auto es una buena elección o no en base a todas las características descriptas  
Evalúo la información de cada columna, viendo el tipo de dato y si poseen datos nulos:

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   price     1727 non-null   object
 1   maint     1727 non-null   object
 2   doors     1727 non-null   object
 3   persons   1727 non-null   object
 4   lug_boot  1727 non-null   object
 5   safety    1727 non-null   object
 6   decision  1727 non-null   object
dtypes: object(7)
memory usage: 47.3+ KB


Todas las columnas son de tipo object (texto), por lo que se debe transformar estos datos a numéricos para que puedan ser tratados por el algoritmo.  
Primero observo todos los valores que posee cada columna:

In [3]:
for e in df.columns:
    print(f'{e}: {df[e].unique()}')

price: ['vhigh' 'high' 'med' 'low']
maint: ['vhigh' 'high' 'med' 'low']
doors: ['2' '3' '4' '5more']
persons: ['2' '4' 'more']
lug_boot: ['small' 'med' 'big']
safety: ['med' 'high' 'low']
decision: ['unacc' 'acc' 'vgood' 'good']


A estos valores de cada columna le paso su respectivo valor numérico ordenado en forma ascendente:

In [4]:
df['price'].replace(('low', 'med', 'high', 'vhigh'), (1,2,3,4), inplace=True)
df['maint'].replace(('low', 'med', 'high', 'vhigh'), (1,2,3,4), inplace=True)
df['doors'].replace(('2', '3', '4', '5more'), (1,2,3,4), inplace=True)
df['persons'].replace(('2', '4', 'more'), (1,2,3), inplace=True)
df['lug_boot'].replace(('small', 'med', 'big'), (1,2,3), inplace=True)
df['safety'].replace(('low', 'med', 'high'), (1,2,3), inplace=True)
df['decision'].replace(('unacc', 'acc', 'good', 'vgood'), (1,2,3,4), inplace=True)

El Dataset con los datos ya convertidos a valores numéricos quedaría de la siguiente manera:

In [5]:
df.head()

,price,maint,doors,persons,lug_boot,safety,decision
0,4,4,1,1,1,2,1
1,4,4,1,1,1,3,1
2,4,4,1,1,2,1,1
3,4,4,1,1,2,2,1
4,4,4,1,1,2,3,1


# División y Procesamiento de Datos

In [6]:
X = df.iloc[:, 0:6].values #Primeras 5 columnas
y = df.iloc[:, 6].values #Última columna

# Modelo de Machine Learning - Tipo supervisado : Árbol de decisión

Al trabajar con un algoritmo de aprendizaje, se tienen que dividir los datos, recomendado:
- 80% aprendizaje
- 20% prueba

In [7]:
from sklearn import tree, metrics #Árbol, metricas para ver el nivel de aprendizaje
from sklearn.model_selection import train_test_split, cross_val_score #Para dividir los datos

X_Train, X_Test, y_Train, y_Test = train_test_split(X, y, test_size=0.2, random_state=0) #%20 de prueba

Definición y entrenamiento del algoritmo:

In [8]:
tr = tree.DecisionTreeClassifier(max_depth=10)
tr.fit(X_Train, y_Train)

DecisionTreeClassifier(max_depth=10)

Predicción sobre los datos de prueba:

In [9]:
y_pred = tr.predict(X_Test)

Porcentaje de aprendizaje:

In [10]:
score = tr.score(X_Test, y_Test)
print('Score: %0.3f' % (score*100))

Score: 96.821


# Conclusión:
El modelo de Árbol de Decisión aplicado al DataSet posee una eficacia del 96.8% siendo un buen porcentaje para la predicción de toma de decisión a la hora de describir un auto.   
Test:

In [11]:
tr.predict([[4,4,3,2,1,2], [2,1,1,2,2,3]])

array([1, 3], dtype=int64)

Al pasarle al modelo 2 ejemplos de autos:  
El primero con **vhigh** de precio, **vhigh** de mantenimiento, **4** puertas, **4** personas, **small** de tamaño del maletero, **med** de seguridad, la predicción es de **unacc** siendo el valor más bajo para la decisión  
El segundo ejemplo al pasarle **med** de precio, **low** de costo de mantenimiento, **2** puertas, **4** personas, **med** de tamaño del maletero y **high** de seguridad. Se obtiene **good** siendo un buen valor para tomar la decisión de comprarlo